In [12]:
import pandas as pd
import numpy as np

In [13]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [14]:
bad_columns = df_train.select_dtypes('object').columns
replacement = {
    'A': 0,
    'B': 1
}

for i in bad_columns:
    df_train[i] = df_train[i].replace(replacement).astype('int32')
    df_test[i] = df_test[i].replace(replacement).astype('int32')

In [15]:
df_train.loc[df_train['HouseYear'] == 20052011, 'HouseYear'] = 2011
df_train.loc[df_train['HouseYear'] > 2020, 'HouseYear'] = 2020
df_train.loc[df_train['LifeSquare'] < 10, 'LifeSquare'] = 10

# Things that I couldn't think of
df_train.loc[df_train['Rooms'].isin([0, 10, 19]), 'Rooms'] = df_train.Rooms.median()
df_train.loc[df_train['HouseFloor'] == 0, 'HouseFloor'] = df_train['HouseFloor'].median()
df_train.loc[df_train['KitchenSquare'] < 3, 'KitchenSquare'] = np.nan
df_train.loc[df_train['KitchenSquare'] > 1000 , 'KitchenSquare'] = np.nan

In [16]:
# At this moment I found that GradientBoostingRegressor pretty much wins the idea from models that I could find. I want to see how much can I go from processed data
# (Shameless copy+paste, since I didn't manage to understand the basics of improving data)

import random

train_df = df_train

square_med_diff = (train_df.loc[train_df['LifeSquare'].notnull(), 'Square']
                   - train_df.loc[train_df['LifeSquare'].notnull(), 'LifeSquare']
                   - train_df.loc[train_df['LifeSquare'].notnull(), 'KitchenSquare']).median()

train_df.loc[train_df['LifeSquare'].isnull(), 'LifeSquare'] = (
    train_df.loc[train_df['LifeSquare'].isnull(), 'Square']
    - train_df.loc[train_df['LifeSquare'].isnull(), 'KitchenSquare']
    - square_med_diff
)

df_train = train_df

In [17]:
for df in (df_train, df_test):
    for column in df.columns:
        df_col = df[column]
        df[column] = df_col.fillna(df_col.median())

    df.set_index('Id')

In [18]:
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Testing all models
classifiers = [
    #svm.SVR(),
    #linear_model.SGDRegressor(),
    #linear_model.BayesianRidge(),
    #linear_model.LassoLars(),
    #linear_model.ARDRegression(),
    #linear_model.PassiveAggressiveRegressor(),
    #linear_model.TheilSenRegressor(),
    #linear_model.LinearRegression(),
    ensemble.GradientBoostingRegressor()]

In [19]:
x = df_train.loc[:, df_train.columns != 'Price']
y = df_train['Price']

x_train, x_val, y_train, y_val = train_test_split(x, y, shuffle=True)

In [20]:
gbr = None

for item in classifiers:
    print(item)
    try:
        clf = item
        clf.fit(x_train, y_train)
        pred = clf.predict(x_val)
        print(r2_score(y_val, pred))
        if isinstance(clf, ensemble.GradientBoostingRegressor):
            gbr = clf
    except Exception as e:
        print(e)

GradientBoostingRegressor()
0.7404015929160459


In [21]:
# Since all the models failed besides GradientBoostingRegressor, the successor for the task is this one
result = clf.predict(df_test)

In [22]:
out = pd.DataFrame({
    'Id': df_test['Id'],
    'Price': result
}).set_index('Id')

out.to_csv('Final_result.csv')